In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys


In [3]:
categorical_columns = ['GENUS', 'TRAPTYPE', 'TRAPSET', 'TRAPCOLLECT']
numerical_columns = ['uvIndex12', 'uvIndex5', 'uvIndex6', 'uvIndex10','uvIndex14', 'uvIndex13','uvIndex11','temperatureMax12','pressure13','humidity6','humidity12','humidity14','humidity11','pressure14','precipIntensityMaxTime12','humidity13']

print(numerical_columns)

['uvIndex12', 'uvIndex5', 'uvIndex6', 'uvIndex10', 'uvIndex14', 'uvIndex13', 'uvIndex11', 'temperatureMax12', 'pressure13', 'humidity6', 'humidity12', 'humidity14', 'humidity11', 'pressure14', 'precipIntensityMaxTime12', 'humidity13']


In [5]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/BerkeleyResults/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [6]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [7]:
def train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(categorical_embedding_sizes, train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.L1Loss()
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_c_t, train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_c_t, test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [8]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('2 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

2 Layers Architecture
MAE =  4.67514967918396 r2 =  0.18449849880986816 SE =  0.01137730647534718 lr =  0.01 hz =  50 dr =  0.0
Best so far 4.67514967918396

MAE =  4.731036949157715 r2 =  0.17733311932811502 SE =  0.017355422527249902 lr =  0.001 hz =  50 dr =  0.0
MAE =  5.712202644348144 r2 =  0.1328155628412358 SE =  0.028487686898439373 lr =  0.0001 hz =  50 dr =  0.0
MAE =  4.71676287651062 r2 =  0.14792101921760767 SE =  0.013474663911229118 lr =  0.01 hz =  60 dr =  0.0
MAE =  4.74684100151062 r2 =  0.18122688319769648 SE =  0.014173059964380886 lr =  0.001 hz =  60 dr =  0.0
MAE =  5.6843212127685545 r2 =  0.05281466605027779 SE =  0.02630310199359016 lr =  0.0001 hz =  60 dr =  0.0
MAE =  4.7001011848449705 r2 =  0.15342561482312395 SE =  0.009870666607678717 lr =  0.01 hz =  70 dr =  0.0
MAE =  4.774170637130737 r2 =  0.13865938865605024 SE =  0.014170368212036266 lr =  0.001 hz =  70 dr =  0.0
MAE =  5.525944709777832 r2 =  0.14223534935129076 SE =  0.02274214627989348 lr =

In [9]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [10]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture
MAE =  4.696956491470337 r2 =  0.12512891970501636 SE =  0.009714677006330491 lr =  0.01 hz =  50 dr =  0.0
Best so far 4.696956491470337

MAE =  4.788709545135498 r2 =  0.1517839158542434 SE =  0.014406636377547005 lr =  0.001 hz =  50 dr =  0.0
MAE =  5.748751449584961 r2 =  0.03782217035985415 SE =  0.022174544940518803 lr =  0.0001 hz =  50 dr =  0.0
MAE =  4.695509576797486 r2 =  0.12627678747929555 SE =  0.014641375903543469 lr =  0.01 hz =  60 dr =  0.0
Best so far 4.695509576797486

MAE =  4.671313667297364 r2 =  0.1719172292462557 SE =  0.012797751516799924 lr =  0.001 hz =  60 dr =  0.0
Best so far 4.671313667297364

MAE =  5.792720127105713 r2 =  0.008877469098489096 SE =  0.02175448743210253 lr =  0.0001 hz =  60 dr =  0.0
MAE =  4.695736074447632 r2 =  0.17903590488168244 SE =  0.00726117753604788 lr =  0.01 hz =  70 dr =  0.0
MAE =  4.738985013961792 r2 =  0.143057384820651 SE =  0.013768274757279608 lr =  0.001 hz =  70 dr =  0.0
MAE =  5.8315023422

In [11]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))
    
    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [12]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  4.663139867782593 r2 =  0.16167074194791514 SE =  0.012917294625841815 lr =  0.01 hz =  50 dr =  0.0
Best so far 4.663139867782593

MAE =  4.8458864212036135 r2 =  0.13209467645363984 SE =  0.01952228197671518 lr =  0.001 hz =  50 dr =  0.0
MAE =  5.714656162261963 r2 =  0.09543401844338945 SE =  0.022212251659719153 lr =  0.0001 hz =  50 dr =  0.0
MAE =  4.701378536224365 r2 =  0.13820230762405195 SE =  0.010345625941306663 lr =  0.01 hz =  60 dr =  0.0
MAE =  4.7513548851013185 r2 =  0.11972375083447855 SE =  0.016546875845216344 lr =  0.001 hz =  60 dr =  0.0
MAE =  5.844612979888916 r2 =  0.023531550652150447 SE =  0.02174494457193169 lr =  0.0001 hz =  60 dr =  0.0
MAE =  4.675617504119873 r2 =  0.16079220697512525 SE =  0.011241052276139165 lr =  0.01 hz =  70 dr =  0.0
MAE =  4.739334297180176 r2 =  0.17063424936921182 SE =  0.011417869375651021 lr =  0.001 hz =  70 dr =  0.0
MAE =  5.618679714202881 r2 =  0.06887102293426742 SE =  0.0250847963259206